In [3]:
#1
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Utilizando dispositivo: {device}")

if device == "cuda":
    num_gpus = torch.cuda.device_count()
    print(f"Número de GPUs disponíveis: {num_gpus}")
    for i in range(num_gpus):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("Nenhuma GPU CUDA encontrada.")

Utilizando dispositivo: cuda
Número de GPUs disponíveis: 2
GPU 0: NVIDIA GeForce RTX 2080 Ti
GPU 1: NVIDIA GeForce RTX 2080 Ti


In [16]:
# 2 – Converter os Arquivos .npy para Imagens
import os
import numpy as np
from PIL import Image

# Define o diretório base onde estão os arquivos .npy e onde as imagens serão salvas
base_dir = "/home/thalles.fontainha/dataset/OAI-MRI-3DDESS"

# Caminhos dos arquivos .npy (ajuste os nomes se necessário)
normal_npy = os.path.join(base_dir, "normal-3DESS-128-64.npy")
abnormal_npy = os.path.join(base_dir, "abnormal-3DESS-128-64.npy")

# Verifica se os arquivos existem
for file in [normal_npy, abnormal_npy]:
    if not os.path.exists(file):
        raise FileNotFoundError(f"Arquivo não encontrado: {file}")
print("Arquivos .npy encontrados!")

# Carrega os arrays
print("Carregando array 'normal'...")
normal_array = np.load(normal_npy)
print(f"Array 'normal' carregado: shape {normal_array.shape}")

print("Carregando array 'abnormal'...")
abnormal_array = np.load(abnormal_npy)
print(f"Array 'abnormal' carregado: shape {abnormal_array.shape}")

# Cria os diretórios para treino e validação (organizados em subpastas por classe)
for split in ["train", "val"]:
    for cls in ["normal", "abnormal"]:
        dir_path = os.path.join(base_dir, split, cls)
        os.makedirs(dir_path, exist_ok=True)
        print(f"Diretório verificado/criado: {dir_path}")

# Função para salvar as imagens a partir de um array numpy
def salvar_imagens(np_array, cls_name):
    n_images = np_array.shape[0]
    n_train = int(0.8 * n_images)  # 80% para treino, 20% para validação
    for i in range(n_images):
        img_data = np_array[i]
        
        # Se a imagem tem 3 dimensões mas o número de canais não é 1, 3 ou 4, reduza para 2D pela média
        if img_data.ndim == 3:
            if img_data.shape[-1] not in [1, 3, 4]:
                img_data = img_data.mean(axis=-1)
        
        # Normaliza a imagem para o intervalo 0-255
        img_norm = ((img_data - img_data.min()) / (img_data.ptp() + 1e-8) * 255).astype(np.uint8)
        try:
            img = Image.fromarray(img_norm)
        except Exception as ex:
            raise ValueError(f"Erro ao converter array para imagem na amostra {i}: {ex}")
        
        # Determina se a imagem vai para treino ou validação
        split_dir = "train" if i < n_train else "val"
        save_dir = os.path.join(base_dir, split_dir, cls_name)
        img_filename = os.path.join(save_dir, f"{cls_name}_{i:04d}.png")
        img.save(img_filename)
        if i % 50 == 0:
            print(f"Salvou imagem: {img_filename}")

print("Salvando imagens da classe 'normal'...")
salvar_imagens(normal_array, cls_name="normal")

print("Salvando imagens da classe 'abnormal'...")
salvar_imagens(abnormal_array, cls_name="abnormal")

print("Conversão dos arquivos .npy para imagens concluída!")

Arquivos .npy encontrados!
Carregando array 'normal'...
Array 'normal' carregado: shape (1659, 128, 128, 64)
Carregando array 'abnormal'...
Array 'abnormal' carregado: shape (1317, 128, 128, 64)
Diretório verificado/criado: /home/thalles.fontainha/dataset/OAI-MRI-3DDESS/train/normal
Diretório verificado/criado: /home/thalles.fontainha/dataset/OAI-MRI-3DDESS/train/abnormal
Diretório verificado/criado: /home/thalles.fontainha/dataset/OAI-MRI-3DDESS/val/normal
Diretório verificado/criado: /home/thalles.fontainha/dataset/OAI-MRI-3DDESS/val/abnormal
Salvando imagens da classe 'normal'...
Salvou imagem: /home/thalles.fontainha/dataset/OAI-MRI-3DDESS/train/normal/normal_0000.png
Salvou imagem: /home/thalles.fontainha/dataset/OAI-MRI-3DDESS/train/normal/normal_0050.png
Salvou imagem: /home/thalles.fontainha/dataset/OAI-MRI-3DDESS/train/normal/normal_0100.png
Salvou imagem: /home/thalles.fontainha/dataset/OAI-MRI-3DDESS/train/normal/normal_0150.png
Salvou imagem: /home/thalles.fontainha/dataset